<a href="https://colab.research.google.com/github/romulo-souza/IA/blob/main/Aprendizado_De_Maquina_Supervisionado/algoritmo_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Exemplo de implementação KNN (K-nearest neighbors ou K vizinhos mais próximos)
(DOCUMENTAÇÃO)
* https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

Base de Dados: Iris.csv (baixar e adicionar em arquivos no google colab)
* https://www.kaggle.com/datasets/uciml/iris?select=Iris.csv

O KNN é muito utilizado em problemas de classificação. Em resumo, o KNN tenta classificar cada amostra de um conjunto de dados avaliando sua distância em relação aos vizinhos mais próximos. Se os vizinhos mais próximos forem majoritariamente de uma classe, a amostra em questão será classificada nesta categoria.

In [ ]:
#Importando a biblioteca pandas
import pandas as pd

In [ ]:
#carrega o arquivo Iris.csv para um DataFrame do pandas.
data = pd.read_csv('Iris.csv')

In [ ]:
data.head() #visualizar as cinco primeiras linhas

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Pré-processamento
* Conversão dos dados atributo classe (species) para dado numérico, para melhor aderência com os classificadores de aprendizado de máquina, que trabalham com equaçoes, funçoes, etc. (Não se adaptam muito bem a valores categóricos(texto))
* LabelEncoder: -> transforma as categorias distintas de um determinado atributo em dados numéricos inteiros sequencial
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
* Remoção do atributo Id, pois nesse caso é um atributo irrelevante, no sentido de descobrir determinado padrão
* Padronizar os dados em uma mesma escala (olhar os intervalos de valores de cada coluna (atributo), caso estejam muito distintos é necessario padronizar para evitar viés no algoritmo de classificação, por exemplo um atributo com intervalo maior pode ser predominante sobre os demais, o que gera uma falsa interpretação para o classificador)

**OBS.1**: **Padronização (StandardScaler)**: Usar quando os dados têm distribuições diferentes ou quando o algoritmo é sensível à variação de escala (ex.: KNN, SVM, redes neurais). Preferível quando os dados possuem outliers e uma distribuição assimétrica;

**Normalização (MinMaxScaler)**: Usar quando você precisa que os dados fiquem em um intervalo fixo (0-1, por exemplo). Útil para redes neurais e quando os dados estão em um intervalo pré-definido.

**OBS.2**: A padronização/normalização deve ser aplicada apenas às colunas que são originalmente numéricas na base de dados. As colunas categóricas, mesmo após serem transformadas em números inteiros pelo LabelEncoder, não devem ser padronizadas/normalizadas. Isso ocorre porque o LabelEncoder atribui números inteiros arbitrários às categorias, mas esses números não têm uma ordem ou escala significativa. Padronizar/Normalizar esses dados pode introduzir distorções que não fazem sentido para o modelo.


In [ ]:
#Remoção da coluna ID
data.drop(['Id'], axis = 1, inplace = True) #axis = 1 pois é coluna, inplace = true -> remoção de forma permanente no dataframe original

In [ ]:
data.columns


Index(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [ ]:
#Transformação da classe para discreto
from sklearn.preprocessing import LabelEncoder # dentro da biblioteca scikit-learn pegamos a subclasse preprocessing, e dentro dessa subclasse importamos o LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
#coluna a ser transformada -> "Species"
data['Species'] = le.fit_transform(data['Species']) # o fit_transform pega o padrao dos dados (quantas categorias distintas tem) e já define quais numeros serão associados a essas categorias. Aplica essa transfomação na propria coluna Species

In [ ]:
data['Species'] #teste para verificar se deu certo a transformação
#data.info() -> verificar os tipos de dados de cada coluna


,Species
0,0
1,0
2,0
3,0
4,0
...,...
145,2
146,2
147,2
148,2


### Rodando o algoritmo sem normalizar primeiramente para verificarmos o impacto gerado
* Algoritmos de distancias entre as amostras mais proximas tem uma super sensibilidade com a questao da escala dos dados


### Divisão da base de dados entre treino e teste -> método holdout
* y - Obtém os valores da classe (estrutura de dados series do panda). Nesse caso a classe seria a coluna 'Species'
* X - Obtém os dados de treinamento (atributos previsores)

In [ ]:
#Biblioteca para separação treino e teste
from sklearn.model_selection import train_test_split #abordagem holdout

In [ ]:
X = data.drop(['Species'], axis=1, inplace = False) #nesse caso inplace = False pois nao queremos remover essa coluna permanentemente do dataframe original 'data', apenas fazemos uma cópia desse dataframe com essa coluna removida para X. Esse inplace = False pode ser omitido.
y = data['Species']

In [ ]:
X #verificando X

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [ ]:
y #verificando y

,Species
0,0
1,0
2,0
3,0
4,0
...,...
145,2
146,2
147,2
148,2


In [ ]:
data #dataframe original

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [ ]:
#Separar em treino e teste (Holdout)
#train_test_split devolve quatro valores
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size = 0.7)  #separando os conjuntos para treino e teste. y_train é gabarito (classes) de X_train, e y_test é gabarito (classes) de X_test
#train_size = 0.7 -> 70% para treino e 30% para teste (consenso)

In [ ]:
#Importar a classe KNeighborsClassifier
#importar da subclasse neighbors de sklearn a classe KNeighborsClassifier para usarmos o KNN
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#gerar o objeto dessa classe KNeighborsClassifier
knn = KNeighborsClassifier(metric='euclidean') #nesse caso estamos apaenas mexendo no parametro de métrica para o calcula das distancia, nesse caso é euclideana. O número de vizinho que o alg. utiliza por default é 5.

In [ ]:
#treinar o algoritmo com o conjunto de treino (função fit)
knn.fit(X_train, y_train) #treina com o conjunto de treino (previsores) 'X_train' e seu gabarito 'y_train'

KNeighborsClassifier(metric='euclidean')

In [ ]:
# Pegar a classe predita (rotulada) pelo classificador usando o conjunto teste, o qual ele não sabe o seu gabarito. Será com base no que ele treinou.
y_pred = knn.predict(X_test) #armazena sua predição em uma lista

### Avaliando o modelo
* avaliar o quanto ele acertou comparando a sua predição 'y_pred' com o gabarito real 'y_test'. Para isso utilizamos o métrica de acurácia

In [ ]:
#utilizando a métrica acurácia para avaliar -> quantos acertos o alg. obteve
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, y_pred) #parametros -> (gabrito real, dados preditos)

0.9333333333333333

### Avaliando com dados Normalizados (IDEAL)

In [ ]:
from sklearn.preprocessing import StandardScaler #importa o StandardScaler que faz uma padronização nos dados, ajustando-os para que tenham média 0 e desvio padrão 1 (z-score). Isso não limita os dados entre 0 e 1, mas os transforma para uma distribuição com essas características.

In [ ]:
#instanciando um objeto da classe StandardScaler
scaler = StandardScaler()

In [ ]:
data.columns

Index(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [ ]:
#definir quais colunas(atributos) serão normalizadas. As colunas selecionadas para serem normalizadas vao sobrepor as colunas originais (não normalizadas)
data[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']] = scaler.fit_transform(data[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']])

In [ ]:
data #verificando essa nova escala

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,-0.900681,1.032057,-1.341272,-1.312977,0
1,-1.143017,-0.124958,-1.341272,-1.312977,0
2,-1.385353,0.337848,-1.398138,-1.312977,0
3,-1.506521,0.106445,-1.284407,-1.312977,0
4,-1.021849,1.263460,-1.341272,-1.312977,0
...,...,...,...,...,...
145,1.038005,-0.124958,0.819624,1.447956,2
146,0.553333,-1.281972,0.705893,0.922064,2
147,0.795669,-0.124958,0.819624,1.053537,2
148,0.432165,0.800654,0.933356,1.447956,2


In [ ]:
#Rodando novamente com os dados normalizados
X = data.drop(['Species'], axis=1, inplace = False)
y = data['Species']
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size = 0.7)

In [ ]:
#treinando novamente o KNN com os dados normalizados
knn.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean')

In [ ]:
#predição feita com os dados normalizados
y_pred = knn.predict(X_test)

In [ ]:
#verificar a acurácia com os dados normalizados
accuracy_score(y_test, y_pred)


0.9555555555555556